In [1]:
import h5py
import numpy as np
import momma_dragonn
import keras
import tensorflow as tf
from avutils import util
from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import History

In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

In [ ]:
from BinaryArchs import get_reg_model
from BinaryArchs import get_rc_model
from BinaryArchs import get_siamese_model
from RunBinaryArchs import get_generators

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [4]:
def get_auROC_auPRC(model, val_data):
    prediction = model.predict(val_data.X)
    auROC = roc_auc_score(val_data.Y, prediction)
    auPRC = average_precision_score(val_data.Y, prediction)
    return auROC, auPRC

In [10]:
class AuRocCallback(keras.callbacks.Callback):
    def __init__(self, model, valid_X, valid_Y):
        self.model = model
        self.valid_X = valid_X
        self.valid_Y = valid_Y
        self.best_auroc_sofar = 0.0
        self.best_weights = None
        self.best_epoch_number = 0
    def on_epoch_end(self, epoch, logs):
        preds = self.model.predict(self.valid_X)
        auroc = roc_auc_score(y_true=self.valid_Y, y_score=preds)
        if (auroc > self.best_auroc_sofar):
            self.best_weights = self.model.get_weights()
            self.best_epoch_number = epoch

In [14]:
def train_model(model, curr_seed, train_data_loader, batch_generator,
                valid_data, epochs_to_train_for, upsampling):

    np.random.seed(curr_seed)
    tf.set_random_seed(curr_seed)
    
    if upsampling == False:
        early_stopping_callback = keras.callbacks.EarlyStopping(
                                  monitor='val_loss',
                                  patience=epochs_to_train_for,
                                  restore_best_weights=True)

        auroc_callback = AuRocCallback(model = model,
                                       valid_X=valid_data.X,
                                       valid_Y=valid_data.Y)
        history = History()
        loss_history = model.fit_generator(train_data_loader.get_batch_generator(),
                                           validation_data = (valid_data.X, valid_data.Y),
                                           epochs=epochs_to_train_for,
                                           steps_per_epoch=50,
                                           class_weight={0:1, 1: 16.5},
                                           callbacks=[auroc_callback, early_stopping_callback, history])
        return early_stopping_callback, auroc_callback, history, model
    else: 
        auroc_callback = AuRocCallback(model = model,
                                       valid_X=valid_data.X,
                                       valid_Y=valid_data.Y)
        history = History()
        loss_history = model.fit_generator(batch_generator,
                                           validation_data = (valid_data.X, valid_data.Y),
                                           epochs=epochs_to_train_for,
                                           steps_per_epoch=50,
                                           callbacks=[auroc_callback, history])
        return auroc_callback, history, model

In [ ]:
parameters = {
    'filters': 16, 
    'input_length':1000, 
    'pool_size':40, 
    'strides': 20
}

In [16]:
epochs_to_train_for=160

np.random.seed(1234)
tf.set_random_seed(1234)
standard_train_batch_generator = get_generators(dataset = "CTCF", 
                                                seq_len = 1000, 
                                                curr_seed = curr_seed, 
                                                is_aug = False)

model = get_rc_model(parameters = parameters, is_weighted_sum = False, use_bias = True)

auroc_callback, history, model = train_model(model = model, 
                                             curr_seed = 1234, 
                                             train_data_loader = None,
                                             batch_generator = standard_train_batch_generator,
                                             valid_data = valid_data, 
                                             epochs_to_train_for = epochs_to_train_for, 
                                             upsampling = True)
model.set_weights(auroc_callback.best_weights)
auROC, auPRC = get_auROC_auPRC(model, valid_data)

curr seed: 1535
Reading in positive bed file
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Got 37196  coords in positive bed file
Reading in negative bed file
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Got 200742  coords in negative bed file
The target proportion of positives of 0.2 requires the negative set to be subsampled by a factor of 2 which will result in a #neg of 100371
Using an offset of  0  before striding






/home/hannah/binary_prediction/binary_model_archs.py:139: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  input_shape=(1000,4), nb_filter=16, filter_length=15))
/home/hannah/binary_prediction/binary_model_archs.py:143: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))


/home/hannah/binary_prediction/binary_model_archs.py:147: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:151: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=20, pool_size=40)`
  pool_length = parameters['pool_size'], strides = parameters['strides']))
/home/hannah/binary_prediction/binary_model_archs.py:161: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(trainable=True, use_bias=True, units=1, kernel_initializer="glorot_uniform")`
  init="glorot_uniform", use_bias = use_bias))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/160





50/50 [==============================] - 122s 2s/step - loss: 0.7145 - acc: 0.6928 - val_loss: 0.5665 - val_acc: 0.7536
Epoch 2/160
50/50 [==============================] - 63s 1s/step - loss: 0.5868 - acc: 0.7288 - val_loss: 0.5555 - val_acc: 0.7571
Epoch 3/160
50/50 [==============================] - 55s 1s/step - loss: 0.5763 - acc: 0.7282 - val_loss: 0.5468 - val_acc: 0.7527
Epoch 4/160
50/50 [==============================] - 48s 952ms/step - loss: 0.5235 - acc: 0.7578 - val_loss: 0.4495 - val_acc: 0.7987
Epoch 5/160
50/50 [==============================] - 41s 811ms/step - loss: 0.4069 - acc: 0.8246 - val_loss: 0.3508 - val_acc: 0.8561
Epoch 6/160
50/50 [==============================] - 36s 713ms/step - loss: 0.3403 - acc: 0.8576 - val_loss: 0.2985 - val_acc: 0.8809
Epoch 7/160
50/50 [==============================] - 32s 640ms/step - loss: 0.2918 - acc: 0.8864 - v

/home/hannah/binary_prediction/binary_model_archs.py:139: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  input_shape=(1000,4), nb_filter=16, filter_length=15))
/home/hannah/binary_prediction/binary_model_archs.py:143: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:147: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:151: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=20, pool_size=40)`
  pool_length = parameters['pool_size'], strides = parameters['strides']))
/home/hannah/binary_prediction/binary_model_archs.py:161: UserWarning: Update your `Dense` call to the Kera

...

Epoch 1/160
50/50 [==============================] - 9s 172ms/step - loss: 1.2383 - acc: 0.6358 - val_loss: 0.5563 - val_acc: 0.7576
Epoch 2/160
50/50 [==============================] - 5s 104ms/step - loss: 0.5826 - acc: 0.7296 - val_loss: 0.5536 - val_acc: 0.7576
Epoch 3/160
50/50 [==============================] - 5s 104ms/step - loss: 0.5791 - acc: 0.7294 - val_loss: 0.5502 - val_acc: 0.7576
Epoch 4/160
50/50 [==============================] - 5s 106ms/step - loss: 0.5734 - acc: 0.7298 - val_loss: 0.5464 - val_acc: 0.7574
Epoch 5/160
50/50 [==============================] - 5s 105ms/step - loss: 0.5646 - acc: 0.7294 - val_loss: 0.5402 - val_acc: 0.7573
Epoch 6/160
50/50 [==============================] - 5s 106ms/step - loss: 0.5617 - acc: 0.7314 - val_loss: 0.5349 - val_acc: 0.7578
Epoch 7/160
50/50 [==============================] - 5s 104ms/step - loss: 0.5569 - acc: 0.7310 - val_loss: 0.5195 - val_acc: 0.7619
Epoch 8/160
50/50 [==============================] - 5s 104ms/step - 

/home/hannah/binary_prediction/binary_model_archs.py:139: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  input_shape=(1000,4), nb_filter=16, filter_length=15))
/home/hannah/binary_prediction/binary_model_archs.py:143: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:147: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:151: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=20, pool_size=40)`
  pool_length = parameters['pool_size'], strides = parameters['strides']))
/home/hannah/binary_prediction/binary_model_archs.py:161: UserWarning: Update your `Dense` call to the Kera

Epoch 1/160
50/50 [==============================] - 9s 177ms/step - loss: 0.7408 - acc: 0.6898 - val_loss: 0.5552 - val_acc: 0.7576
Epoch 2/160
50/50 [==============================] - 5s 106ms/step - loss: 0.5840 - acc: 0.7294 - val_loss: 0.5547 - val_acc: 0.7576
Epoch 3/160
50/50 [==============================] - 5s 103ms/step - loss: 0.5801 - acc: 0.7296 - val_loss: 0.5502 - val_acc: 0.7576
Epoch 4/160
50/50 [==============================] - 5s 106ms/step - loss: 0.5817 - acc: 0.7296 - val_loss: 0.5519 - val_acc: 0.7576
Epoch 5/160
50/50 [==============================] - 5s 103ms/step - loss: 0.5769 - acc: 0.7290 - val_loss: 0.5462 - val_acc: 0.7575
Epoch 6/160
50/50 [==============================] - 5s 105ms/step - loss: 0.5676 - acc: 0.7308 - val_loss: 0.5298 - val_acc: 0.7572
Epoch 7/160
50/50 [==============================] - 5s 107ms/step - loss: 0.5517 - acc: 0.7352 - val_loss: 0.5138 - val_acc: 0.759137 - a
Epoch 8/160
50/50 [==============================] - 5s 105ms/s

/home/hannah/binary_prediction/binary_model_archs.py:139: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  input_shape=(1000,4), nb_filter=16, filter_length=15))
/home/hannah/binary_prediction/binary_model_archs.py:143: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:147: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  nb_filter=16, filter_length=14))
/home/hannah/binary_prediction/binary_model_archs.py:151: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=20, pool_size=40)`
  pool_length = parameters['pool_size'], strides = parameters['strides']))
/home/hannah/binary_prediction/binary_model_archs.py:161: UserWarning: Update your `Dense` call to the Kera

Epoch 1/160
50/50 [==============================] - 9s 177ms/step - loss: 0.6869 - acc: 0.6824 - val_loss: 0.5502 - val_acc: 0.7530
Epoch 2/160
50/50 [==============================] - 5s 100ms/step - loss: 0.5540 - acc: 0.7354 - val_loss: 0.5127 - val_acc: 0.7641
Epoch 3/160
50/50 [==============================] - 5s 100ms/step - loss: 0.5009 - acc: 0.7716 - val_loss: 0.4295 - val_acc: 0.8115
Epoch 4/160
50/50 [==============================] - 5s 101ms/step - loss: 0.3892 - acc: 0.8400 - val_loss: 0.3236 - val_acc: 0.8731
Epoch 5/160
50/50 [==============================] - 5s 101ms/step - loss: 0.3044 - acc: 0.8782 - val_loss: 0.2698 - val_acc: 0.8985
Epoch 6/160
50/50 [==============================] - 5s 100ms/step - loss: 0.2760 - acc: 0.8984 - val_loss: 0.2430 - val_acc: 0.9090
Epoch 7/160
50/50 [==============================] - 5s 101ms/step - loss: 0.2258 - acc: 0.9204 - val_loss: 0.2212 - val_acc: 0.9193
Epoch 8/160
50/50 [==============================] - 5s 100ms/step - 